# Measuring the total cold gas mass in Aurora's M87 observation
Grant Tremblay (grant.tremblay@yale.edu)

## tl,dr
I get $M_{\mathrm{H}_2} = (9.04 \pm 1.3) \times 10^5~M_\odot$ assuming a Milky Way $X_\mathrm{CO} = $

## The Details

Following (e.g.) Bollato+13, the total mass of H$_2$ scales with the CO(1-0) emission integral $S_\mathrm{CO}\Delta V$ as:

$M_\mathrm{mol}  = \frac{1.05 \times 10^4}{3.2} ~ \left(\frac{X_{\mathrm{CO}}}{2\times 10^{20}~\frac{\mathrm{cm}^{-2}}{\mathrm{K~km~s}^{-1}}}\right) \left( \frac{1}{1+z}\right) \left(\frac{S_{\mathrm{CO}}\Delta v}{\mathrm{Jy~km~s}^{-1}}\right) \left(\frac{D_\mathrm{L}}{\mathrm{Mpc}}\right)^2 M_\odot$ 

We'll assume a Milky Way CO-to-H2 conversion factor

$X_{\mathrm{CO}} = 2 \times 10^{20}~\mathrm{cm}^{-2} \left(\mathrm{K~km~s}^{-1}\right)^{-1}$

and a CO(2-1) to CO(1-0) flux density ratio of $3.2$. Note that these assumptions are metallicity dependent and have important consequences. For a discussion of this relevant to cool core BCGs, see e.g. McNamara+14. 

### Step 1: In CASA v4.5, I fit a Gaussian to the total CO(2-1) line profile

This can be done in ``casaviewer`` interactively. I fit an elliptical region that covers all $>3\sigma$ CO(2-1) emission found in the ALMA beam-corrected map. I then fit a Gaussian to the line extracted from this region. I won't go into how to do that, as it's pretty straightforward and probably evident in the following screenshots. 

<img src="1.png">
<img src="2.png">
<img src="3.png">

### Step 2: Set some basics

In [1]:
import astropy.units as u
import astropy.constants as const

Set cosmology...

In [2]:
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

Make some units...

In [3]:
emission_integral_units_mJy = u.mJy * u.km / u.s
emission_integral_units_Jy = u.Jy * u.km / u.s

xco_unit = u.cm**(-2) * (u.K * u.km * u.s**(-1))**(-1)

### Step 3: Set parameters for M87

In [4]:
##### PUT THE REDSHIFT OF YOUR GALAXY HERE ######

redshift = 0.004283 # Redshift of M87


##### PUT YOUR GAUSSIAN AREA IN mJY * km/s HERE #####

gaussian_area = 817.015
gaussian_area_err = 118.184


##### PUT YOUR CONVERSION FACTOR & RATIO HERE ######

xco = 2.0e20 * xco_unit
ratio = 3.2  # CO(2-1) to CO(1-0) flux density ratio

######################################################




emission_integral = (gaussian_area * emission_integral_units_mJy).to(emission_integral_units_Jy)
emission_integral_err = (gaussian_area_err * emission_integral_units_mJy).to(emission_integral_units_Jy)


### Step 4: Calculate the gas mass...

In [5]:
def gasmass(redshift,xco,ratio,emission_integral):
    xco_mw = 2.0e20 * xco_unit
    ldist = cosmo.luminosity_distance(redshift).to(u.Mpc)
    
    rawmass = 1.05e4 * (xco / xco_mw) * (1 + redshift)**(-1) * (emission_integral / emission_integral_units_Jy) * (ldist/u.Mpc)**2 * (1.0/ratio)
    
    finalmass = rawmass * u.M_sun
    
    
    return finalmass

In [6]:
mass = gasmass(redshift=redshift,xco=xco,ratio=ratio,emission_integral=emission_integral)
mass_err = gasmass(redshift=redshift,xco=xco,ratio=ratio,emission_integral=emission_integral_err)

The total cold gas mass is: 

In [7]:
mass

<Quantity 904124.8466750141 solMass>

$\pm$

In [8]:
mass_err

<Quantity 130784.73575080001 solMass>

So the total mass is

$(9.04 \pm 1.3) \times 10^5~M_\odot$